<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/HyeonminNam/200728_NaiveBayesClassifier(Multiclass).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NaiveBayesClassifier(Multiclass) 클래스화

In [44]:
import pandas as pd
import math

class NBC():
    def __init__(self, word_lst, class_lst):
        self.word_lst = word_lst
        self.class_lst = class_lst
        self.token_lst = []
        self.class_set = []

    # 각 토큰별 클래스 빈도 추출
    def token_dic(self):
        # 토큰과 클래스 유형 리스트 만들기
        token_lst = []
        for x in word_lst:
            token_lst += x.split()
        class_set = list(set(class_lst))
        self.word_lst = word_lst
        self.class_set = class_set
        self.token_lst = list(set(word_lst))

        # 단어별 클래스 빈도 정리한 사전 구축
        token_dic = {}
        for x in self.token_lst:
            token_dic[x] = [0 for i in range(len(self.class_set))]
        for i, row in df.iterrows():
            tokens = row['tokens'].split()
            for n in range(len(self.class_set)):
                if row['class'] == self.class_set[n]:
                    for x in tokens:
                        token_dic[x][n] += 1
        return token_dic

    # 빈도, 확률, 로그확률 데이터프레임 구축
    def freq_df(self, smoothing = 0.5):
        token_dic = self.token_dic()
        df2 = pd.DataFrame(index=self.token_lst, columns=self.class_set)
        for word, cnt in token_dic.items():
            for i, cls in enumerate(self.class_set):
                df2.loc[word][cls] = cnt[i]
            # Laplace smoothing
        for i, cls in enumerate(self.class_set):
            df2['p_w_{}'.format(cls)] = (smoothing + df2[cls]) / (2*smoothing+sum(df2[cls]))
        for i, cls in enumerate(self.class_set):
            df2['log_pw_{}'.format(cls)] = [math.log(x) for x in df2['p_w_{}'.format(cls)]]
        return df2

    # 확률 계산
    def calc(self, input_token, smoothing = 0.5):
        df2 = self.freq_df(smoothing)
        input_token_lst = input_token.split()
        perc_dic = {}
        # 전체 클래스 확률의 log값
        for x in self.class_set:
            perc_dic['p_{}'.format(x)] = math.log(self.class_lst.count(x)/len(self.class_lst))

        # 각 단어별 클래스 확률의 log값 더해주기
        for input_token in input_token_lst:
            for x in self.class_set:
                perc_dic['p_{}'.format(x)] += df2['log_pw_{}'.format(x)][input_token]    
        for x in self.class_set:
            perc_dic['p_{}'.format(x)] = math.exp(perc_dic['p_{}'.format(x)])
        for x in self.class_set:
            percentage = perc_dic['p_{}'.format(x)]/sum(perc_dic.values())
            print('{} 확률: {}%'.format(x, percentage*100))

## 클래스 테스트

In [45]:
nbc = NBC(text, spam)

In [46]:
nbc.token_dic()

{'award': [1, 0],
 'contact': [1, 0],
 'free': [2, 3],
 'get': [0, 1],
 'lottery': [0, 2],
 'me': [1, 1],
 'scholarship': [1, 0],
 'ticket': [0, 1],
 'to': [1, 0],
 'won': [1, 0],
 'you': [2, 2]}

In [47]:
nbc.freq_df()

,normal,spam,p_w_normal,p_w_spam,log_pw_normal,log_pw_spam
award,1,0,0.136364,0.0454545,-1.992430,-3.091042
ticket,0,1,0.0454545,0.136364,-3.091042,-1.992430
contact,1,0,0.136364,0.0454545,-1.992430,-3.091042
me,1,1,0.136364,0.136364,-1.992430,-1.992430
free,2,3,0.227273,0.318182,-1.481605,-1.145132
lottery,0,2,0.0454545,0.227273,-3.091042,-1.481605
won,1,0,0.136364,0.0454545,-1.992430,-3.091042
scholarship,1,0,0.136364,0.0454545,-1.992430,-3.091042
to,1,0,0.136364,0.0454545,-1.992430,-3.091042
you,2,2,0.227273,0.227273,-1.481605,-1.481605


In [48]:
nbc.calc('free lottery', smoothing=0.5)

normal 확률: 12.500000000000009%
spam 확률: 87.49999999999999%
